## Imports

In [1]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")
from gpt import GPT
from intent_model import IntentModel

## Load data

In [2]:
# Load the data from JSON file
with open("../intent_recognition_v2.json", "r") as f:
    data = json.load(f)

train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])
    

## Load models

In [3]:
intent_model = IntentModel(train_data, label_map, labels)

## Check data completeness

In [4]:
from collections import defaultdict

# Your data is already a list of dictionaries
categories = defaultdict(list)

for item in data:
    intent_text = item['intent']
    if not item['use_gpt'] and not item['responses']:

        intent_text += ' (Needs answers)'
    categories[item['category']].append(intent_text)

for category, intents in categories.items():
    if(category == 'privacy_policy' or category ==  'legal_statement'):
        continue

    print(f'Category: {category}')
    for intent in intents:
        print(f'  Intent: {intent}')
    print('\n')

Category: generic
  Intent: hello_response
  Intent: thanks_response




## Get evaluation data

In [5]:
evaluation_file = "./evaluation_results.json"

if os.path.isfile(evaluation_file):
    with open(evaluation_file, "r") as file:
        accuracies = json.load(file)
else:
    accuracies = intent_model.evaluate_model(data)

## Check data correctness

In [6]:
print("total correctness", accuracies["total_correctness"])

# "total_correct_counts": 608,
# "total_counts": 663,

for evaluation in accuracies["results"]:
    if (evaluation["total_correctness"] == 100):
        continue

    print(f"Intent: {evaluation['correct_intent']}")
    print(f"Train question results: {evaluation['train_questions_results']}")

    test_results = evaluation.get('test_questions_results')
    if test_results:
        print(f"Test question results: {test_results}")
        
    advanced_results = evaluation.get('test_questions_advanced_results')
    if advanced_results:
        print(f"Advanced test question results: {advanced_results}")
    
    print(f"Total correctness for all questions: {evaluation['total_correctness']}%")
    print("\n")


total correctness 0.0
Intent: summarization_privacy_policy
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: data_sharing_opt_out
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: third_parties
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: data_collection
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: personal_information
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: data_storage
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: personal_information_rights
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: access_personal_data
Train question results: 0 out of 10 correct
Total correctness for all questions: 0.0%


Intent: update_personal_data
Train question res